## WTH is this crap?

Idk man it's just an attempt to have fun with (crappy) codes

## References

1. [Investopedia](http://i.investopedia.com/inv/pdf/tutorials/dcf.pdf)
2. [Put another link here](https://.com)
3. [Another one](https://.com)

## Intro

#### #np led zeppelin's 'when the levee breaks'

$$ DCF = \frac{FCFF_1}{(1 + r) ^ 1} + \frac{FCFF_2}{(1 + r) ^ 2} + ... + \frac{FCFF_n}{(1 + r) ^ n} $$

Where $FCFF$ stands for unlevered free cash flow for a given year ($FCFF_1$ means free cash flow in year 1, and so on); and $r$ stands for the discount rate.

$ FCFF = EBITDA - CAPEX - \Delta NWC - tax\_expenses $

Where:
1. EBITDA = earnings before interest, depreciation, and amortization, or pretax income,
2. CAPEX = capital expenditure, or fixed assets,
3. $ \Delta $ NWC = changes in net working capital, which can be derived by subtracting total current assets and total current liabilities in current and previous years,
4. tax_expenses is already self-explanatory (hopefully).

## Load Data

In [1]:
import os
import pandas
import requests
import string
from functools import partial, reduce
from bs4 import BeautifulSoup

In [2]:
PATH = os.getcwd() + '/sample-output/'

In [3]:
def get_single_dataframe(year):
    
    file_name = [x for x in os.listdir('sample-output/') if x.endswith('{}.csv'.format(year))]
    
    dfs = []
    for f in file_name:
        dfs.append(pandas.read_csv(PATH + f))
        
    return reduce(partial(pandas.merge, on = 'ticker_code'), dfs)

In [4]:
def get_multiple_dataframes(year):
    
    dfs = []
    for year in (year - 1, year):
        dfs.append(get_single_dataframe(year))
        
    return pandas.concat(dfs, sort = False).reset_index(drop = True)

In [5]:
df = get_multiple_dataframes(2018)

In [6]:
df.head()

,cash_and_equivalents,account_receivables_third_party,account_receivables_related_party,total_current_assets,fixed_assets,total_non_current_assets,total_assets,total_current_liabilities,total_non_current_liabilities,total_equity,...,sales_and_admin_expenses,depreciation_expenses,pretax_income,tax_expenses,after_tax_income_operational,after_tax_income_nonoperational,after_tax_income,comprehensive_income,income_per_share,year
0,2.622920e+05,7.130200e+04,4.762360e+05,4.245730e+06,1.015222e+07,2.068970e+07,24935426,2.309417e+06,4.089571e+06,18536438,...,7.564180e+05,2.938505e+06,-8.248760e+05,2113629,NaN,2113629,2064015,1044.50000,NaN,2017
1,4.756197e+10,7.892476e+10,3.741202e+09,1.480236e+11,4.008374e+10,2.802024e+11,428225997056,1.224256e+11,1.870374e+11,118763007361,...,1.355039e+11,-3.260841e+10,-4.391221e+09,-36999632465,NaN,-36999632465,-12307881645,-10.08000,NaN,2017
2,5.055008e+08,NaN,NaN,NaN,9.379421e+07,NaN,2966605878,NaN,NaN,1375352652,...,2.568367e+08,1.650197e+08,-4.197568e+06,160822141,NaN,160822141,199028319,259.00000,NaN,2017
3,1.406905e+08,1.134228e+08,2.608860e+07,3.394791e+08,6.028107e+08,7.031947e+08,1042673806,2.125897e+08,6.677611e+08,162323025,...,6.851247e+07,1.189273e+07,-8.093967e+06,3798762,NaN,3798762,2945255,0.00202,NaN,2017
4,9.022280e+11,4.270049e+10,4.713665e+10,3.358272e+12,3.592293e+11,1.070568e+12,4428840550479,4.782086e+11,4.402101e+11,3510421847790,...,5.640819e+11,9.599803e+11,-1.792935e+11,780686814661,NaN,780686814661,741795436411,45.62000,NaN,2017


## Calculate Free Cash Flow

In [7]:
def get_free_cash_flow(year):
    
    # select necessary variables
    # sort by ticker code in ascending order
    # reset index
    df_nwc = df[['year', 'ticker_code', 'total_current_assets', 'total_current_liabilities']]\
    .sort_values(by=['ticker_code'], ascending=True)\
    .reset_index(drop=True)
    
    # add net working capital as a column
    df_nwc['net_working_capital'] = df_nwc['total_current_assets'] - df_nwc['total_current_liabilities']
    
    # changes in net working capital
    df_nwc['net_working_capital_delta'] = df_nwc\
    .groupby('ticker_code')['net_working_capital']\
    .diff()
    
    # filter dataframe only in year latest year
    df_nwc = df_nwc[df_nwc['year'] == year][['ticker_code', 'net_working_capital_delta']]\
    .reset_index(drop=True)
    
    # calculate free cash flow
    df_cf = df[df['year'] == 2018][['year', 'ticker_code', 'pretax_income', 'fixed_assets', 'tax_expenses']]
    
    df_cf = pandas\
    .merge(df_cf, df_nwc, how = 'inner', on = 'ticker_code')\
    .reset_index(drop = True)
    
    df_cf['free_cash_flow'] = df_cf['pretax_income'] \
    - df_cf['fixed_assets'] \
    - df_cf['net_working_capital_delta'] \
    - df_cf['tax_expenses']
    
    return df_cf[['year', 'ticker_code', 'free_cash_flow']].reset_index(drop=True)

In [8]:
df_cf = get_free_cash_flow(2018)

In [9]:
df_cf.head()

,year,ticker_code,free_cash_flow
0,2018,AALI,-11913878.0
1,2018,ABBA,NaN
2,2018,ABDA,NaN
3,2018,ABMM,NaN
4,2018,ACES,NaN


### Woooo we're half way there, whooa oah...

## Calculate Discount Rate (r)

What we need to find out:
1. Cost of Equity 
2. Cost of Debt

### Cost of Equity

Cost of equity, or RE, can obtained by calculating [Capital Asset Pricing Model](https://www.investopedia.com/terms/c/capm.asp). Following are the items:

1. Risk-free rate
2. Beta
3. Equity market risk premium (market rate subtracted by risk-free rate)

After getting the items, we need to calculate the cost of equity as follows:

$$ RE = risk\_free\_rate + beta * (market\_rate - risk\_free\_rate) $$

### Risk-free rate

Risk-free rate is commonly derived from T-bills rate. In Indonesia, the equivalent is [Obligasi Negara](https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx), which is traded in the secondary market. Currently there are many traded bonds with various coupon rates, so I am going to use the average rate. First, we need to scrape the table to get the coupon rate. 

In [10]:
def get_risk_free_rate():
    
    # get bonds rate from Bank Central of Indonesia
    # attempt thrice in case of connection failure
    i = 0
    while i < 3:
        try:
            url = requests.get('https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx')
            if url.status_code == 200:
                soup = BeautifulSoup(url.text, 'html')
            i = 3
        except Exception:
            i += 1
    
    # find table which contains the bonds rate
    table = soup.find('table', {'class': 'table1'})
    trs = table.find_all('tr')
    
    # get the bonds rate
    tds = []
    for t in trs:
        try:
            tds.append(t.find_all('td')[-1]\
                       .text\
                       .replace(' ', ''))
        except IndexError:
            pass
        
    for t in range(len(tds)):
        if '\r' in tds[t]:
            tds[t] = tds[t].replace('\r', '')
        elif '\n' in tds[t]:
            tds[t] = tds[t].replace('\n', '')
        try:
            tds[t] = float(tds[t]) / 100
        except ValueError:
            pass
        
    risk_free_rate = []
    for t in tds:
        if type(t) == float:
            risk_free_rate.append(t)
    
    return sum(risk_free_rate) / len(risk_free_rate)

In [11]:
def get_cost_of_equity(risk_free_rate, beta, market_rate):
    
    return risk_free_rate + beta * (market_rate - risk_free_rate)

### Cost of Debt

Cost of debt, or symbolized as RD, can be calculated as follows:

$$ RD = 1 - tax\_rate $$

According to [Trading Economics](https://tradingeconomics.com/indonesia/corporate-tax-rate), Indonesia's corporate tax rate is 25%. Hence, the cost of equity is 75%.

### Capital Structure (Weighted Average Cost of Capital)

In [12]:
def get_wacc():

    # get capital structure
    df_wacc = df[df['year'] == 2018][['ticker_code', 'total_assets', 'total_equity']] \
    .reset_index(drop = True)
    
    df_wacc['equity_proportion'] = df_wacc['total_equity'] / df_wacc['total_assets']
    df_wacc['liabilities_proportion'] = 1 - df_wacc['equity_proportion']
    
    # cost of equity
    re = get_cost_of_equity(
        risk_free_rate=get_risk_free_rate(),
        beta=1,
        market_rate=0.05
    )
    
    # cost of debt
    rd = 1 - 0.25
    
    # weighted cost of capital
    df_wacc['wacc'] = (re * df_wacc['equity_proportion']) + (rd * (1 - df_wacc['equity_proportion']))
    df_wacc = df_wacc[['ticker_code', 'wacc']]
    
    return df_wacc

In [13]:
df_wacc = get_wacc()

In [14]:
df_wacc.head()

,ticker_code,wacc
0,AALI,0.242416
1,ABBA,0.479925
2,ABDA,0.426835
3,ABMM,0.546373
4,ACES,0.192825


In [15]:
df_cf = pandas.merge(df_cf, df_wacc, how = 'inner', on = 'ticker_code')

In [16]:
df_cf.head()

,year,ticker_code,free_cash_flow,wacc
0,2018,AALI,-11913878.0,0.242416
1,2018,ABBA,NaN,0.479925
2,2018,ABDA,NaN,0.426835
3,2018,ABMM,NaN,0.546373
4,2018,ACES,NaN,0.192825


## Calculate Fair Value

What we need to find out is the terminal value

$$ terminal\_value = \frac{FPYCF * (1 + LTCFGR)}{r - LTCFGR} $$

Jeez, what's that?!?!?!

1. FPYCF = Final projected year cash flow
2. LTCGR = Long-term cash flow growth rate
3. r = discounted rate

In [17]:
def get_fair_value(rate=0.0001, num_year=1, year=2018):
    
    """
    Returns fair value.
    
    Arguments:
        - rate = cash flow growth rate. By default, assume there is no growth.
        - num_year = number of projected years. By default, this only projects for the next one year.
        - year = ...
    """
    
    df_cf['terminal_value'] = (df_cf['free_cash_flow'] * (1 + (rate * num_year)) \
                            * (1 + rate) \
                            / (df_cf['wacc'] - rate))
    
    
    df_cf['enterprise_value'] = df_cf['free_cash_flow'] / ((1 + rate) ** num_year)
    
    df_cf['fair_value'] = df_cf['terminal_value'] - df_cf['enterprise_value']
    
    return df_cf[['year', 'ticker_code', 'fair_value']]

In [18]:
df_cf = get_fair_value()

## Finalleee!

In [19]:
# see companies which have fair value >= 0
# does that mean good or bad? idk
df_cf[df_cf['fair_value'] >= 0].reset_index(drop = True)

,year,ticker_code,fair_value
0,2018,APOL,1.226486e+11
1,2018,BIMA,5.649683e+09
2,2018,CMPP,2.129053e+10
3,2018,CNKO,5.879183e+07
4,2018,CSIS,1.684100e+11
5,2018,EMTK,2.099725e+09
6,2018,ENRG,1.585314e+07
7,2018,ERAA,1.743606e+12
8,2018,ETWA,3.947004e+10
9,2018,FORU,7.832467e+09


### To Do

1. Get fair value per current share price
2. Label whether a company is over- or under-valued
3. Find out whether the numbers make heckin' sense or not
4. Go to bed and have a sleep!!!!!!!!!!!!!!